In [1]:
import os
import pandas as pd
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt

import sklearn
import sklearn.metrics
import sklearn.model_selection
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [5]:
# explore ResNet feature matrices
image_folder = "train_input/resnet_features/"

# function to load folder into arrays and then it returns that same array
def loadImages(path):
    image_files = sorted([os.path.join(path, file)
         for file in os.listdir(path) if file.endswith('.npy')])
    return image_files

In [3]:
def get_average_features(filenames):
    """Load and aggregate the resnet features by the average.

    Args:
        filenames: list of filenames of length `num_patients` corresponding to resnet features

    Returns:
        features: np.array of mean resnet features, shape `(num_patients, 2048)`
    """
    # Load numpy arrays
    features = []
    for f in filenames:
        patient_features = np.load(f)

        # Remove location features (but we could use them?)
        patient_features = patient_features[:, 3:]

        aggregated_features = np.mean(patient_features, axis=0)
        features.append(aggregated_features)

    features = np.stack(features, axis=0)
    return features

In [4]:
# load feature npy folder into arrays and returns that same array of strings
def loadFiles(path):
    feature_files = sorted([os.path.join(path, file)
         for file in os.listdir(path) if file.endswith('.npy')])
    return feature_files

In [6]:
# precise training set and test set relative location

train_dir = Path("train_input/resnet_features")
test_dir = Path("test_input/resnet_features")

train_output_filename = Path("training_output.csv")

train_output = pd.read_csv(train_output_filename)

<h2>Use locally annotated information</h2>

In [7]:
# old id looks like: ID_387_annotated_tile_0_15_69_30.jpg
# reformat the annotation id for upcoming df joining
# function for label file

def get_new_id(old_id):
    # get rid of the .jpg extension
    old_id = Path(old_id).stem
    # store all characters in a list
    string_list = old_id.split('_')
    # new_id looks like: <patient_id>_<zoom_level>_<x_coord>_<y_coord>
    new_id = f"{string_list[0]}_{string_list[1]}_{string_list[-3]}_{string_list[-2]}_{string_list[-1]}"
    return new_id

In [8]:
# load local annotations (tile-level)
# id goes as follows: <patient_id>_annotated_tile_<tile_id>_<tile_coords>
local_annot = pd.read_csv('train_input/train_tile_annotations.csv')
local_annot.rename(columns={'Unnamed: 0': 'Tile_annotation_id'}, inplace=True)

# add new column new_id
local_annot['new_tile_id'] = local_annot['Tile_annotation_id'].map(get_new_id)

In [23]:
# function to load 11 annotated npy files in resnet features folder
def loadAnnotatedData(path):
    feature_files = sorted([os.path.join(path, file)
         for file in os.listdir(path) if file.endswith('_annotated.npy')])
    return feature_files


# compile all locally annotated data into a dataframe to form strong supervised dataset
def dataCompiler(filelist):
    df_data = pd.DataFrame()
    
    # Load numpy arrays
    for f in filelist:
        try:
            patient_features = np.load(f)
            patient_id = Path(f).stem.strip("_annotated")

            # add patient id to features
            df_patient = pd.DataFrame(data=patient_features)
            df_patient['patient_id'] = patient_id

            # add df to global dataframe
            df_data = df_data.append(df_patient, ignore_index=True)
        
        except FileNotFoundError:
            pass
    
    # rename dataframe with proper column names
    colnames = ['zoom_level', 'x_coord', 'y_coord'] + [i for i in range(1,2049)] + ['patient_id']
    df_data.columns = colnames
    
    return df_data


# generate new_id for annotated data
def generate_new_id(patient_id, zoom, x, y):
    element_list = [patient_id, str(int(zoom)), str(int(x)), str(int(y))]
    separator = "_"
    new_id = separator.join(element_list)
    return new_id

In [12]:
# gives the list of annotated patient npy files
annotatedFiles_train = loadAnnotatedData(train_dir)

# complete annotated dataset
annotatedData = dataCompiler(annotatedFiles_train)

# add new column new_tile_id
annotatedData['new_tile_id']:str = annotatedData.apply(lambda x: generate_new_id(x.patient_id, x.zoom_level, x.x_coord, x.y_coord),  axis = 1)

In [13]:
# join the features table and label table with newly created new_tile_id
data = annotatedData.merge(local_annot, how='inner', on='new_tile_id')

# Separate now local features and target for models
# local features for train (tile-level)
local_features = np.array(data.iloc[:, 3:2051])

# local targets (tile-level)
local_labels = data["Target"].values

<h2>Apply SVM to local annotated data</h2>

In [14]:
# Get filenames for train
filenames_train = loadFiles(train_dir)

# Get global labels (patient-wise) for train
labels_train = train_output["Target"].values

# check if the number of observations and labels corresponds
assert len(filenames_train) == len(labels_train)

In [15]:
# Get the numpy filenames for test
filenames_test = loadFiles(test_dir)
# ID list without its suffix (ex: "ID_005")
ids_test = [Path(f).stem for f in filenames_test]

In [16]:
# Get the resnet features and aggregate them by the average
features_train = get_average_features(filenames_train)
features_test = get_average_features(filenames_test)

In [24]:
# given path of a filename, returns a numpy array
def get_tile_features(filename):
    # Load npy to numpy arrays 
    patient_features = np.load(filename)
    
    # Remove location features (but we could use them?)
    patient_features = patient_features[:, 3:]
    return patient_features

In [19]:
# Evaluate SVM model without C parameter

# number of runs for cross-validation
num_runs = 3
# number of splits for cross-validation
num_splits = 5

# Multiple cross validations on the local feature training set
aucs = []
accuracies =[]
recalls = []

# Standardize training features
scaler = StandardScaler()
train_X = scaler.fit_transform(local_features)

for seed in range(num_runs):
    #Create a SVM Classifier without C parameter
    clf = svm.SVC(kernel='linear')

    cv = sklearn.model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)

    # Cross validation on the training set
    auc = sklearn.model_selection.cross_val_score(clf, X=train_X, y=local_labels,
                                                  cv=cv, scoring="roc_auc", verbose=0)
    accuracy = sklearn.model_selection.cross_val_score(clf, X=train_X, y=local_labels,
                                                  cv=cv, scoring="accuracy", verbose=0)
    recall = sklearn.model_selection.cross_val_score(clf, X=train_X, y=local_labels,
                                                  cv=cv, scoring="recall", verbose=0)
    
    aucs.append(auc)
    accuracies.append(accuracy)
    recalls.append(recall)

aucs = np.array(aucs)
accuracies = np.array(accuracies)
recalls = np.array(recalls)

print("Predicting strong labels by tile-level resnet")
print("AUC: mean {}, std {}".format(aucs.mean(), aucs.std()))
print("Accuracy: mean {}, std {}".format(accuracies.mean(), accuracies.std()))
print("True Positive Rate: mean {}, std {}".format(recalls.mean(), recalls.std()))

Predicting strong labels by tile-level resnet
AUC: mean 0.919714839559539, std 0.01773988538222485
Accuracy: mean 0.9578557068267216, std 0.004225995867068012
True Positive Rate: mean 0.714234342223554, std 0.044187437945193704


In [20]:
# Evaluate SVM models with hyperparameter C

# Test using the following values for coefficient 'c'
c_coeff = np.array([5**-3, 5**-2, 5**-1, 1 , 5 , 5**2, 5**3])

# number of runs for cross-validation
num_runs = 3
# number of splits for cross-validation
num_splits = 5

# Multiple cross validations on the local feature training set
aucs = []
accuracies =[]
recalls = []

# Standardize training features
scaler = StandardScaler()
train_X = scaler.fit_transform(local_features)

for i in c_coeff:
    # create a SVM classifier with various C parameter values
    svf = SVC(C=i, kernel='linear')

    cv = sklearn.model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)

    # Cross validation on the training set
    auc = sklearn.model_selection.cross_val_score(svf, X=train_X, y=local_labels,
                                              cv=cv, scoring="roc_auc", verbose=0)
    accuracy = sklearn.model_selection.cross_val_score(svf, X=train_X, y=local_labels,
                                              cv=cv, scoring="accuracy", verbose=0)
    recall = sklearn.model_selection.cross_val_score(svf, X=train_X, y=local_labels,
                                              cv=cv, scoring="recall", verbose=0)

    print(f"When c_coeff is {i}")
    print("AUC: mean {}, std {}".format(auc.mean(), auc.std()))
    print("Accuracy: mean {}, std {}".format(accuracy.mean(), accuracy.std()))
    print("True Positive Rate: mean {}, std {}".format(recall.mean(), recall.std()))

When c_coeff is 0.008
AUC: mean 0.9409161675829241, std 0.009161927498755718
Accuracy: mean 0.9711573220123946, std 0.005224424078312489
True Positive Rate: mean 0.7156128258915194, std 0.04897567906589284
When c_coeff is 0.04
AUC: mean 0.9227796457698807, std 0.018378565926560293
Accuracy: mean 0.9599958522422289, std 0.006192905181682128
True Positive Rate: mean 0.7198281889921087, std 0.06031469059858214
When c_coeff is 0.2
AUC: mean 0.9185350830270054, std 0.02124387175568656
Accuracy: mean 0.9584154101400477, std 0.0062590471388418055
True Positive Rate: mean 0.7184197382878833, std 0.0665063473717086
When c_coeff is 1.0
AUC: mean 0.9185350830270054, std 0.02124387175568656
Accuracy: mean 0.9584154101400477, std 0.0062590471388418055
True Positive Rate: mean 0.7184197382878833, std 0.0665063473717086
When c_coeff is 5.0
AUC: mean 0.9185350830270054, std 0.02124387175568656
Accuracy: mean 0.9584154101400477, std 0.0062590471388418055
True Positive Rate: mean 0.7184197382878833, std

<h2>Grid Search for SVM with RBF Kernel</h2>

In [ ]:
# It is usually a good idea to scale the data for SVM training.
scaler = StandardScaler()
X = scaler.fit_transform(local_features)

# Train classifiers
# For an initial search, a logarithmic grid with basis
# 10 is often helpful. Using a basis of 2, a finer
# tuning can be achieved but at a much higher cost.

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = sklearn.model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
grid = sklearn.model_selection.GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, local_labels)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

<p>Finally the grid search of RBF takes too long to find the optimal hyperparameters. Since the data dimension is relatively high, theoractically there is no need for data projection and the linear kernel can reach comparable performance. </p>

<h2>Create added training set with unlabeled positive cases</h2>

In [24]:
# retrieve all positive patient case
positive_patients = train_output[train_output['Target']==1]
positive_patients['npy_ID'] = positive_patients.apply(lambda x: f"ID_{str(x['ID']).zfill(3)}.npy", axis=1)

# store all positive case npys to an array for compilation
added_pat_list = np.array(positive_patients['npy_ID'])
added_pat_list = [f"train_input/resnet_features/{i}" for i in added_pat_list]

# Compile unlabeled positive cases in training set
posData = dataCompiler(added_pat_list)

# add new column new_tile_id
posData['new_tile_id']:str = posData.apply(lambda x: generate_new_id(x.patient_id, x.zoom_level, x.x_coord, x.y_coord),  axis = 1)
    
# create a training test set for prediction
added_features = np.array(posData.iloc[:, 3:2051])

<ipython-input-24-77232071a7d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_patients['npy_ID'] = positive_patients.apply(lambda x: f"ID_{str(x['ID']).zfill(3)}.npy", axis=1)


<h2>Train SVM model with tuned hyperparameter and predict on the added unlabeled set</h2>

In [25]:
# Standardize training and test features
scaler = StandardScaler()
train_X = scaler.fit_transform(local_features)
test_X = scaler.transform(added_features)

# Use the tile-level resnet features to predict the labels
# Create a svm Classifier
clf = svm.SVC(C=0.008, kernel='linear', probability=True)

# Train the model using the training sets
clf.fit(train_X, local_labels)

# Predict the response for test dataset
y_pred = clf.predict(test_X)
y_pred_proba = clf.predict_proba(test_X)

In [26]:
# recap prediction info to dataframe
recap = pd.DataFrame(data=y_pred_proba)
recap['Target'] = y_pred
recap['patient_id'] = posData['patient_id']
recap.columns = ['negative', 'positive', 'Target', 'ID']

# aggregation: count number of positive and negative cases for each patient
recap_count = pd.pivot_table(recap, index=['ID'], columns=['Target'], aggfunc={'Target': 'count'})
recap_count.columns = ['NB_neg', 'NB_pos']

<h2>Top Scoring example selection on predicted data and form new training set</h2>

In [27]:
# check if patient-level data to be eliminated
eliminated = recap_count[recap_count['NB_pos'].isnull()]

# all global categorical predictions should point to positive => NB_pos shouldn't be NaN
print(f"There is {len(eliminated)} wrong categorical prediction(s).")

# create list of patient_ID to eliminated
eliminated.reset_index(inplace=True)
eli_list = list(eliminated['ID'])

There is 2 wrong categorical prediction(s).


In [28]:
# Top Scoring example selection 
# to form a simulated dataset to complete the old locally annotated one 
posData['Target'] = y_pred
posData['positive_proba'] = recap['positive']
neg_examples = posData[posData['positive_proba']<0.003]
pos_examples = posData[posData['positive_proba']>=0.95]
selected_examples = pd.concat([neg_examples, pos_examples], ignore_index=True)

In [29]:
# eliminate those samples in the list from the data set
selected_examples = selected_examples[~selected_examples['patient_id'].isin(eli_list)]

# form simulated dataset from unlabeled data
simulated_features = np.array(selected_examples.iloc[:, 3:2051])
simulated_labels = selected_examples['Target'].values

# add the simulated features and labels to labelised training data
# to form a bigger training set
new_features = np.append(local_features, simulated_features, axis=0)
new_labels = np.append(local_labels, simulated_labels, axis=0)
assert new_features.shape[0] == new_labels.shape[0]

<h2>Take newly-formed dataset as input and predict on the test set</h2>

In [ ]:
# Evaluate the new model trained with new dataset

# number of runs for cross-validation
num_runs = 3
# number of splits for cross-validation
num_splits = 5

# Multiple cross validations on the local feature training set
aucs = []
accuracies =[]
recalls = []

# Standardize training and test features and apply standardization to test features
scaler = StandardScaler()
train_X = scaler.fit_transform(new_features)

for seed in range(num_runs):
    #Create a svm Classifier
    clf = svm.SVC(C=0.008, kernel='linear', probability=True) # Linear Kernel without C parameter

    cv = sklearn.model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)

    # Cross validation on the training set
    auc = sklearn.model_selection.cross_val_score(clf, X=train_X, y=new_labels,
                                                  cv=cv, scoring="roc_auc", verbose=0)
    accuracy = sklearn.model_selection.cross_val_score(clf, X=train_X, y=new_labels,
                                                  cv=cv, scoring="accuracy", verbose=0)
    recall = sklearn.model_selection.cross_val_score(clf, X=train_X, y=new_labels,
                                                  cv=cv, scoring="recall", verbose=0)
    
    aucs.append(auc)
    accuracies.append(accuracy)
    recalls.append(recall)

aucs = np.array(aucs)
accuracies = np.array(accuracies)
recalls = np.array(recalls)

print("Predicting strong labels by tile-level resnet")
print("AUC: mean {}, std {}".format(aucs.mean(), aucs.std()))
print("Accuracy: mean {}, std {}".format(accuracies.mean(), accuracies.std()))
print("True Positive Rate: mean {}, std {}".format(recalls.mean(), recalls.std()))

In [ ]:
# Predict on test set

# empty df to store output of all patients
df_output = pd.DataFrame()

# Standardize training and test features and apply standardization to test features
scaler = StandardScaler()
train_X = scaler.fit_transform(new_features)

# Use the tile-level resnet features to predict the labels
# Create a svm Classifier
clf = svm.SVC(C=0.008, kernel='linear', probability=True)

# Train the model using the training sets
clf.fit(train_X, new_labels)

for f in filenames_test: 
    patient_id:str = Path(f).stem.split("ID_")[1]
    test_X = get_tile_features(f)
    
    # do the PCA transformation on test set
    test_X = scaler.transform(test_X)
    
    # Predict the response for test dataset
    y_pred = clf.predict(test_X)
    y_pred_proba = clf.predict_proba(test_X)[:, 1] #keep only positive probability
    
    # Check that predictions are in [0, 1]
    assert np.max(y_pred_proba) <= 1.0
    assert np.min(y_pred_proba) >= 0.0
    
    test_output = pd.DataFrame({"ID": patient_id, "Target": y_pred_proba, "Category": y_pred})
    df_output = df_output.append(test_output, ignore_index=True)

In [231]:
# if positive proba is absent, ie. no positive class detected for a patient
# choose negative proba. Otherwise, use positive proba
def select_final_proba(pos_proba, nega_proba):
    if np.isnan(pos_proba):
        final_proba = nega_proba
    else:
        final_proba = pos_proba
    return final_proba

# create recap table with aggregated information
recap = pd.pivot_table(df_output, values='Target', index=['ID'], columns=['Category'], aggfunc={'Target': np.mean})
# rename recap table columns
colnames = ['negative_proba', 'positive_proba']
recap.columns = colnames

# apply final proba selection at patient-level and create new column
recap['Target'] = recap.apply(lambda x: select_final_proba(x.positive_proba, x.negative_proba), axis=1)

# drop useless columns and save result to csv
output = recap.drop(columns=['negative_proba', 'positive_proba'])
output.to_csv("predictions/SVM_with_simulated_labels.csv")